In [1]:
! pip install bs4

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1264 sha256=f8e38e601546dd3ca125423e2d35578f3dc22257e541289602dd921399002f4c
  Stored in directory: c:\users\lige2\appdata\local\pip\cache\wheels\d4\c8\5b\b5be9c20e5e4503d04a6eac8a3cd5c2393505c29f02bea0960
Successfully built bs4


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import regex as re
import urllib

In [3]:
def parse_main(URL, links, category):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table')

    for link in table.find_all('a'):
        if str(link.get('href')).endswith('.htm') == True:
            link_j = urllib.parse.urljoin('https://wwwn.cdc.gov/', link.get('href'))
            links[category].append(link_j)


urls = {'DM':'https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Demographics&CycleBeginYear=2013',
        'DIET':'https://wwwn.cdc.gov/nchs/nhanes/Search/DataPage.aspx?Component=Dietary&CycleBeginYear=2013',
        'EXAM':'https://wwwn.cdc.gov/nchs/nhanes/Search/DataPage.aspx?Component=Examination&CycleBeginYear=2013',
        'LAB':'https://wwwn.cdc.gov/nchs/nhanes/Search/DataPage.aspx?Component=Laboratory&CycleBeginYear=2013',
        'QUES':'https://wwwn.cdc.gov/nchs/nhanes/Search/DataPage.aspx?Component=Questionnaire&CycleBeginYear=2013'}

links = {v:[] for v in urls.keys()}

for c, URL in urls.items():
    print(c, URL)
    parse_main(URL, links, c)

DM https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Demographics&CycleBeginYear=2013
DIET https://wwwn.cdc.gov/nchs/nhanes/Search/DataPage.aspx?Component=Dietary&CycleBeginYear=2013
EXAM https://wwwn.cdc.gov/nchs/nhanes/Search/DataPage.aspx?Component=Examination&CycleBeginYear=2013
LAB https://wwwn.cdc.gov/nchs/nhanes/Search/DataPage.aspx?Component=Laboratory&CycleBeginYear=2013
QUES https://wwwn.cdc.gov/nchs/nhanes/Search/DataPage.aspx?Component=Questionnaire&CycleBeginYear=2013


In [4]:
def parse_nhanes(links, codes):
    for c, URLs in links.items():
        for URL in URLs:
            # access webs site
            page = requests.get(URL)

            # parse data
            soup = BeautifulSoup(page.content, 'html.parser')
            containers = soup.find_all('dl')
            for i in containers:
                try:
                    varname = str(i.find("dt",string="Variable Name: ").findNext("dd").text)
                    saslabel = str(i.find("dt",string="SAS Label: ").findNext("dd").text)
#                     print(varname, saslabel)
                    codes['category'].append(c)
                    codes['variable'].append(varname.strip())
                    codes['label'].append(saslabel.strip())
                except:
#                     print(f'error in {URL} {i}')
                    pass
    return codes

codes = {"category": [], "variable": [], "label": []}


parse_nhanes(links, codes)


codebook = pd.DataFrame(codes)

In [5]:
codebook.value_counts()

category  variable  label                                 
LAB       SEQN      Respondent sequence number                91
QUES      SEQN      Respondent sequence number                43
EXAM      SEQN      Respondent sequence number                19
          DXXPT30X  x-coordinates of outline points 31        13
          DXXPT35Y  y-coordinates of outline points 36        13
                                                              ..
          OHX11CJS  LOA: Max L(C) MF FGM-CEJ(mm)               1
          OHX11CSC  Coronal Caries:  Surface condition #11     1
          OHX11CTC  Coronal Caries: Tooth Count #11            1
          OHX11LAA  LOA: Max L(C) ML calculated AL(mm)         1
QUES      WTSVOC2Y  VOC Subsample Weight                       1
Name: count, Length: 4087, dtype: int64

In [6]:
code_unique = codebook[['variable', 'label']].drop_duplicates(subset=['variable'])
print(code_unique)

      variable                                   label
0         SEQN              Respondent sequence number
1     SDDSRVYR                      Data release cycle
2     RIDSTATR            Interview/Examination status
3     RIAGENDR                                  Gender
4     RIDAGEYR               Age in years at screening
...        ...                                     ...
7302    WHD140  Self-reported greatest weight (pounds)
7303    WHQ150                Age when heaviest weight
7305   WHQ030M         How do you consider your weight
7306    WHQ500               Trying to do about weight
7307    WHQ520          How often tried to lose weight

[4019 rows x 2 columns]


In [7]:
code_unique.to_csv('nhanes_2013_2014_codebook.csv', index=False)